In [0]:
from __future__ import print_function
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [0]:
path = get_file("nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
with io.open(path, encoding="utf-8") as f:
  text = f.read().lower()
print("Corpus lenght: ",len(text))

chars = sorted(list(set(text)))
print("Total Chars: ", len(chars))
#Mapping from char to index
char_indices = dict((c,i) for i,c in enumerate(chars))
#Mapping index char to char
indences_char = dict((i,c) for i,c in enumerate(chars))


Corpus lenght:  600893
Total Chars:  57


In [0]:
maxlen = 40
step = 3
sentence = []
next_chars = []
for i in range(0, len(text)-maxlen, step):
  sentence.append(text[i:i+maxlen])
  next_chars.append(text[i+maxlen])
print("nb sequence: ", len(sentence))

nb sequence:  200285


In [0]:
print("vectorization...")
x = np.zeros((len(sentence), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentence), len(chars)), dtype = np.bool)
for i , sentence in enumerate(sentence):
  for t, char in enumerate(sentence):
    x[i,t,char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

vectorization...


In [0]:
import os
print("Build Model...")
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation="softmax"))

optimizer = RMSprop(lr=0.01)
if(os.path.exists("weight.h5")):
  model.load_weight("weight.h5")
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

W0801 03:33:30.522046 139882501691264 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Build Model...


In [0]:
def sample(preds, temperature=1.0):
  #Help function to sample an index from a probability array
  preds = np.asarray(preds).astype("float64")
  preds = np.log(preds)/ temperature
  exp_preds = np.exp(preds)
  preds = exp_preds/np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
  # Function invoked at end of each epoch. Prints generated text.
  print()
  print('----- Generating text after Epoch: %d' % epoch)

  start_index = random.randint(0, len(text) - maxlen - 1)
  for diversity in [0.2, 0.5, 1.0, 1.2]:
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
          x_pred[0, t, char_indices[char]] = 1.

      preds = model.predict(x_pred, verbose=0)[0]
      next_index = sample(preds, diversity)
      next_char = indices_char[next_index]

      sentence = sentence[1:] + next_char

      sys.stdout.write(next_char)
      sys.stdout.flush()
    print()

In [0]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
model.fit(x, y, batch_size=128, epochs=20, callbacks=[print_callback])
model.save_weights("weights.h5")

Epoch 1/20
200192/200285 [============================>.] - ETA: 0s - loss: 1.4743
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "uth" such valuations,
in spite of their "
uth" such valuations,
in spite of their subvent as the sense of the sense to the instinction of the sension, the highest and intelletous to the will belongs of the sense of the sense of the soul of its all the soul of his soul the sense of the sense to free is not something and as the states of the sense of the sense of the sense of the sense of all the sense of the sense of the sense of the sensible to the sense in the constantime and 
----- diversity: 0.5
----- Generating with seed: "uth" such valuations,
in spite of their "
uth" such valuations,
in spite of their or a determiness, really to the self-tor, and a the that consequent out of the naturation of the contrary to so--like souls of the related to man has the suffering to the higher scholar to mannce is a most demotions

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


in ewidlied lorm and its gorng, and vanion to finds
to "hindanc";--the
kand wern with the way of when the base--and bashing--but to thed zell fore a problem of all it were distrusted us before to anla""; and with it, begrel events hows his pew the brings mereby with portic and
world out" "chi
----- diversity: 1.2
----- Generating with seed: "might be the diffidence of a god among m"
might be the diffidence of a god among maded great about hes
ableg of so
the world  owerifactle exceccult,." obvifucle only mankind caiv prisond calture"
i smulignt he werce---by living an woman, would like not be necess is it, gregarianantctly wishes that well moded."



inthings, german: but
maned and goor: knearn; life implay expredsity, itself stood, corture and
threep us conculual philosoite of "intelews such a phene. "whereccencor
200285/200285 [==============================] - 133s 663us/sample - loss: 1.4580
Epoch 3/20
200192/200285 [============================>.] - ETA: 0s - loss: 1.4439
----- Ge